# Part IV: Modelling

## Basic settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import pickle
import sys
import warnings
from datetime import datetime, timedelta

import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

warnings.filterwarnings("ignore")

# Set plotting style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("deep")

# Set random seed for reproducibility
np.random.seed(2025)

d:\Asus\App\miniconda\workspace\envs\sales_forcast_xai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
src_path = os.path.abspath(os.path.join("../..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [4]:
from utils.plots import plot_forecast_single
from utils.utils import flatten_prophet_predictions, weighted_absolute_percentage_error

In [6]:
import os
import pandas as pd
import numpy as np

DATA_DIR = "../../data"

# 1. LOAD DATA ĐÃ PREPROCESS VÀ FEATURE ENGINEERING
df_sales = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/sales_data_preprocessed.csv"),
    parse_dates=["date"]
)
df_weather = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/weather_preprocessed.csv"),
    parse_dates=["date"]
)
# df_weather_key_store_merged = pd.read_csv(
#     os.path.join(DATA_DIR, "data_processed/weather_key_store_merged.csv"),
#     parse_dates=["date"]
# )

# Đây là file đã có is_kaggle_test và toàn bộ features
df_features = pd.read_feather(os.path.join(DATA_DIR,'data_processed/feature_engineered_data_89_features.feather'))

print("Full feature data:", df_features.shape)
print("Kaggle test rows:", df_features['is_kaggle_test'].sum())
print("Train rows:", (df_features['is_kaggle_test'] == 0).sum())




Full feature data: (686187, 89)
Kaggle test rows: 526917
Train rows: 159270


## Load data

In [7]:
df_features['is_valid'] = 0
mask_train = df_features['is_kaggle_test'] == 0
cutoff_date = pd.Timestamp("2014-08-01")
df_features.loc[mask_train & (df_features['date'] >= cutoff_date), 'is_valid'] = 1

# 2. Tách train/valid và kaggle test
df_train = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 0)].copy()
df_valid = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 1)].copy()
df_kaggle_test = df_features[df_features['is_kaggle_test'] == 1].copy()

print("Final splits:")
print("  Train:", df_train.shape)
print("  Valid:", df_valid.shape)
print("  Kaggle test:", df_kaggle_test.shape)

Final splits:
  Train: (153496, 90)
  Valid: (5774, 90)
  Kaggle test: (526917, 90)


In [8]:
df_kaggle_test.head()

,date,store_nbr,item_nbr,units,logunits,is_kaggle_test,station_nbr,tmax,depart,cool,...,logunits_ewma_14d_a05,logunits_ewma_28d_a05,logunits_ewma_7d_a075,logunits_ewma_14d_a075,logunits_ewma_28d_a075,store_sum_7d,store_mean_7d,item_sum_7d,item_mean_7d,is_valid
159270,2013-04-01,2,1,NaN,NaN,True,14,71.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,8.670772,1.238682,7.203406,1.029058,0
159271,2013-04-01,3,1,NaN,NaN,True,7,68.0,6.2,0.0,...,NaN,NaN,NaN,NaN,NaN,8.825560,1.260794,6.510258,1.085043,0
159272,2013-04-01,6,1,NaN,NaN,True,14,71.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,12.102488,1.728927,5.817111,1.163422,0
159273,2013-04-01,7,1,NaN,NaN,True,6,86.0,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,8.648221,1.235460,5.123964,1.280991,0
159274,2013-04-01,8,1,NaN,NaN,True,4,87.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,9.572480,1.367497,3.178054,1.059351,0


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153496 entries, 0 to 159249
Data columns (total 90 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    153496 non-null  datetime64[ns]
 1   store_nbr               153496 non-null  int64         
 2   item_nbr                153496 non-null  int64         
 3   units                   153496 non-null  float64       
 4   logunits                153496 non-null  float64       
 5   is_kaggle_test          153496 non-null  bool          
 6   station_nbr             153496 non-null  int64         
 7   tmax                    153496 non-null  float64       
 8   depart                  153496 non-null  float64       
 9   cool                    153496 non-null  float64       
 10  sunrise                 153496 non-null  float64       
 11  sunset                  153496 non-null  float64       
 12  snowfall                153496 non-

## Build xboost model

In [10]:
drop_cols = [
    'date', 'units', 'logunits',
    'is_kaggle_test', 'is_valid',
    'store_nbr', 'item_nbr', 'station_nbr',
    'tmax', 'depart', 'cool', 'sunrise', 'sunset',
    'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
    'resultspeed', 'resultdir',
    'BCFG', 'BLDU', 'BLSN', 'BR', 'DU', 'DZ', 'FG', 'FG+',
    'FU', 'FZDZ', 'FZFG', 'FZRA', 'GR', 'GS', 'HZ', 'MIFG',
    'PL', 'PRFG', 'RA', 'SG', 'SN', 'SQ', 'TS', 'TSRA',
    'TSSN', 'UP', 'VCFG', 'VCTS'
]

feature_cols = [c for c in df_features.columns if c not in drop_cols]

### Build a lightgbm model

In [11]:
def create_xgboost_model(df_train, df_valid, feature_cols):
    """
    Building XGBoost models per (store, item) pair with optimized parameters
    """
    print("Building XGBoost models per (store, item)...")

    pairs = df_train[['store_nbr', 'item_nbr']].drop_duplicates()

    xgb_models = {}
    xgb_metrics = []
    all_actual, all_pred = [], []

    for _, row in pairs.iterrows():
        s, i = int(row['store_nbr']), int(row['item_nbr'])

        train_si = df_train[(df_train['store_nbr'] == s) &
                            (df_train['item_nbr'] == i)]
        valid_si = df_valid[(df_valid['store_nbr'] == s) &
                            (df_valid['item_nbr'] == i)]

        # Không đủ dữ liệu thì bỏ
        if len(train_si) < 20:
            continue

        X_tr = train_si[feature_cols]
        y_tr = train_si['logunits']
        X_va = valid_si[feature_cols]
        y_va = valid_si['logunits']

        # Sử dụng tham số tối ưu từ Optuna
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "verbosity": 0,
            "max_depth": 7,
            "learning_rate": 0.010113217909182654,
            "subsample": 0.7997828501729438,
            "colsample_bytree": 0.5574819523335883,
            "colsample_bylevel": 0.6808836497575912,
            "min_child_weight": 10,
            "gamma": 0.19820345140994988,
            "reg_alpha": 5.277969035526557e-06,
            "reg_lambda": 1.3028132129165083e-07,
        }

        # Prepare DMatrix
        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        
        # Train với early stopping nếu có validation set
        if not valid_si.empty:
            dvalid = xgb.DMatrix(X_va, label=y_va)
            model = xgb.train(
                params,
                dtrain,
                num_boost_round=2000,
                evals=[(dvalid, "validation")],
                early_stopping_rounds=100,
                verbose_eval=False
            )
        else:
            model = xgb.train(
                params,
                dtrain,
                num_boost_round=2000,
                verbose_eval=False
            )

        key = (s, i)
        xgb_models[key] = model

        if not valid_si.empty:
            dvalid_pred = xgb.DMatrix(X_va)
            preds = model.predict(dvalid_pred)
            mae = mean_absolute_error(y_va, preds)
            rmse = np.sqrt(mean_squared_error(y_va, preds))
            wape = weighted_absolute_percentage_error(y_va, preds)
            xgb_metrics.append((s, i, mae, rmse, wape))
            all_actual.extend(y_va)
            all_pred.extend(preds)

    if xgb_metrics:
        mae_avg = np.mean([m[2] for m in xgb_metrics])
        rmse_avg = np.mean([m[3] for m in xgb_metrics])
        wape_avg = weighted_absolute_percentage_error(
            np.array(all_actual), np.array(all_pred)
        )
        print(f"\nOverall XGBoost per-(store,item) valid metrics:")
        print(f"MAE: {mae_avg:.4f} | RMSE: {rmse_avg:.4f} | WAPE: {wape_avg:.4f}")

    print("Models built:", len(xgb_models))
    return xgb_models, xgb_metrics

In [12]:
xgb_models, xgb_metrics = create_xgboost_model(
    df_train, df_valid, feature_cols
)

Building XGBoost models per (store, item)...

Overall XGBoost per-(store,item) valid metrics:
MAE: 0.2337 | RMSE: 0.2933 | WAPE: 15.9816
Models built: 255


In [13]:
def create_xgboost_submission(
    df_kaggle_test, xgb_models, feature_cols,
    filename="submission_xgb_per_store_item.csv"
):
    
    df_kaggle = df_kaggle_test.copy()
    kaggle_parts = []

    pairs_test = df_kaggle[['store_nbr', 'item_nbr']].drop_duplicates()

    for _, row in pairs_test.iterrows():
        s, i = int(row['store_nbr']), int(row['item_nbr'])
        key = (s, i)

        test_si = df_kaggle[
            (df_kaggle["store_nbr"] == s) &
            (df_kaggle["item_nbr"] == i)
        ].copy()
        if test_si.empty:
            continue

        X_test_si = test_si[feature_cols]

        if key in xgb_models:
            # XGBoost cần DMatrix để predict
            dtest = xgb.DMatrix(X_test_si)
            yhat_log = xgb_models[key].predict(dtest)
            units = np.expm1(yhat_log)
            units = np.clip(units, 0, None)
        else:
            units = np.zeros(len(test_si))

        test_si["units"] = units
        kaggle_parts.append(test_si)

    if not kaggle_parts:
        raise ValueError("Không có dòng nào được dự đoán.")

    df_kaggle_pred = pd.concat(kaggle_parts, ignore_index=True)

    df_kaggle_pred["date_str"] = df_kaggle_pred["date"].dt.strftime("%Y-%m-%d")
    df_kaggle_pred = df_kaggle_pred.sort_values(["date_str", "store_nbr", "item_nbr"])

    df_kaggle_pred["id"] = (
        df_kaggle_pred["store_nbr"].astype(str) + "_" +
        df_kaggle_pred["item_nbr"].astype(str) + "_" +
        df_kaggle_pred["date_str"]
    )

    submission = df_kaggle_pred[["id", "units"]].reset_index(drop=True)
    submission.to_csv(filename, index=False)
    print(f"✓ Saved {filename} ({len(submission)} rows)")
    return submission

submission = create_xgboost_submission(
    df_kaggle_test, xgb_models, feature_cols,
    filename="submission_xgb_per_store_item.csv"
)

✓ Saved submission_xgb_per_store_item.csv (526917 rows)


In [14]:
# Kiểm tra nhanh: Đếm số lượng dòng dự đoán khác 0
non_zero_preds = submission[submission['units'] > 0]
print(f"Số lượng dòng có dự đoán bán hàng: {len(non_zero_preds)}")
print("Ví dụ 5 dòng có số liệu:")
print(non_zero_preds.head())

Số lượng dòng có dự đoán bán hàng: 25684
Ví dụ 5 dòng có số liệu:
                 id       units
4    2_5_2013-04-01   64.529587
10  2_11_2013-04-01    0.977494
43  2_44_2013-04-01  178.603058
84  2_85_2013-04-01    0.016873
92  2_93_2013-04-01    0.077276


### (Optional) Fine tunning using Optuna

In [16]:
def optimize_xgboost(X_train, y_train, X_valid, y_valid, n_trials=50):
    print("\nOptimizing XGBoost model with Optuna...")
    
    def objective(trial):
        # Hyperparameters search space for XGBoost
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "verbosity": 0,
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "n_estimators": 2000,
        }
        
        # Prepare DMatrix
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)
        
        # Train with early stopping
        evals_result = {}
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=2000,
            evals=[(dvalid, "validation")],
            evals_result=evals_result,
            early_stopping_rounds=100,
            verbose_eval=False
        )
        
        # Predict và tính metric mục tiêu (WAPE)
        dvalid_pred = xgb.DMatrix(X_valid)
        preds = model.predict(dvalid_pred)
        # wape = weighted_absolute_percentage_error(y_valid, preds)
        rmse = np.sqrt(mean_squared_error(y_valid, preds))
        return rmse  # Optimize trực tiếp WAPE
    
    # Chạy Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    
    print("\nBest params found:")
    best_params = study.best_params
    best_params.update({
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "verbosity": 0,
        "n_estimators": 2000
    })
    
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    
    # Train final model với best params
    dtrain_final = xgb.DMatrix(X_train, label=y_train)
    dvalid_final = xgb.DMatrix(X_valid, label=y_valid)
    
    evals_result_final = {}
    final_model = xgb.train(
        best_params,
        dtrain_final,
        num_boost_round=2000,
        evals=[(dvalid_final, "validation")],
        evals_result=evals_result_final,
        early_stopping_rounds=100,
        verbose_eval=100
    )
    

    dvalid_pred = xgb.DMatrix(X_valid)
    valid_preds = final_model.predict(dvalid_pred)
    test_mae = mean_absolute_error(y_valid, valid_preds)
    test_rmse = np.sqrt(mean_squared_error(y_valid, valid_preds))
    test_wape = weighted_absolute_percentage_error(y_valid, valid_preds)
    
    print(f"\nOptimized XGBoost Valid Metrics - MAE: {test_mae:.3f}, RMSE: {test_rmse:.3f}, WAPE: {test_wape:.3f}")
    
    return final_model, best_params, (test_mae, test_rmse, test_wape)

In [17]:
optimized_model, best_params, optimized_metrics = optimize_xgboost(
    X_train, y_train, X_valid, y_valid, n_trials=50
)


[I 2025-11-29 04:23:57,870] A new study created in memory with name: no-name-88b9b745-c4fc-43f7-8bc6-00a1f26b879f



Optimizing XGBoost model with Optuna...


[I 2025-11-29 04:24:08,016] Trial 0 finished with value: 0.36189861059052447 and parameters: {'max_depth': 7, 'learning_rate': 0.02017264434054835, 'subsample': 0.6239811888590259, 'colsample_bytree': 0.9549488498433473, 'colsample_bylevel': 0.6865894304709103, 'min_child_weight': 4, 'gamma': 0.5619609314352109, 'reg_alpha': 3.022616652731097e-06, 'reg_lambda': 3.36444740730876e-07}. Best is trial 0 with value: 0.36189861059052447.
[I 2025-11-29 04:24:11,946] Trial 1 finished with value: 0.36106731614377136 and parameters: {'max_depth': 8, 'learning_rate': 0.06891046469929286, 'subsample': 0.7219350229051826, 'colsample_bytree': 0.6926040548892449, 'colsample_bylevel': 0.5240938570767197, 'min_child_weight': 4, 'gamma': 0.7373671768059187, 'reg_alpha': 1.4979158695888615e-08, 'reg_lambda': 0.0005264408443746626}. Best is trial 1 with value: 0.36106731614377136.
[I 2025-11-29 04:24:18,510] Trial 2 finished with value: 0.3636285512508863 and parameters: {'max_depth': 4, 'learning_rate': 


Best params found:
  max_depth: 7
  learning_rate: 0.010113217909182654
  subsample: 0.7997828501729438
  colsample_bytree: 0.5574819523335883
  colsample_bylevel: 0.6808836497575912
  min_child_weight: 10
  gamma: 0.19820345140994988
  reg_alpha: 5.277969035526557e-06
  reg_lambda: 1.3028132129165083e-07
  objective: reg:squarederror
  eval_metric: rmse
  verbosity: 0
  n_estimators: 2000
[0]	validation-rmse:1.71359
[100]	validation-rmse:0.70899
[200]	validation-rmse:0.42783
[300]	validation-rmse:0.37335
[400]	validation-rmse:0.36416
[500]	validation-rmse:0.36209
[600]	validation-rmse:0.36133
[700]	validation-rmse:0.36095
[800]	validation-rmse:0.36075
[900]	validation-rmse:0.36059
[1000]	validation-rmse:0.36046
[1100]	validation-rmse:0.36041
[1200]	validation-rmse:0.36040
[1300]	validation-rmse:0.36023
[1400]	validation-rmse:0.36014
[1500]	validation-rmse:0.36019
[1529]	validation-rmse:0.36010

Optimized XGBoost Valid Metrics - MAE: 0.200, RMSE: 0.360, WAPE: 14.663


In [18]:
submission = create_xgboost_submission(df_kaggle_test, optimized_model, filename="submission_xgboost_optim.csv")

✓ Saved submission_xgboost_optim.csv (526917 rows)


## Evaluating model

In [19]:
def evaluate_model(model, X_test, y_test, data):
    """
    Evaluate the model performance on the test set (last 3 months of 2017)
    """
    print("\nEvaluating model performance on test set...")

    # Make predictions on the test set
    test_preds = model.predict(X_test)

    # Calculate metrics
    test_mae = mean_absolute_error(y_test, test_preds)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
    test_wape = weighted_absolute_percentage_error(y_test, test_preds)

    # Print evaluation results
    print(f"Final Model Test Evaluation:")
    print(f"    MAE: {test_mae:.2f}")
    print(f"    RMSE: {test_rmse:.2f}")
    print(f"    WAPE: {test_wape:.2f}%")

    # Analyze errors by time period (month)
    test_results = data[data["is_test"]].copy()
    test_results["prediction"] = test_preds
    test_results["error"] = test_results["sales"] - test_results["prediction"]
    test_results["abs_error"] = np.abs(test_results["error"])
    test_results["month_name"] = test_results["date"].dt.strftime("%B")

    # Summarize errors by month
    monthly_errors = (
        test_results.groupby("month_name")
        .agg({"abs_error": "mean", "error": "mean", "sales": "mean"})
        .reset_index()
    )
    monthly_errors["error_pct"] = (
        100 * monthly_errors["abs_error"] / monthly_errors["sales"]
    )

    print("\nError Analysis by Month:")
    print(
        monthly_errors[["month_name", "abs_error", "error_pct"]].to_string(index=False)
    )

    # Store results for visualization
    # Include month and store information for granular analysis
    test_results["year_month"] = test_results["date"].dt.strftime("%Y-%m")

    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, test_preds, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "r--")
    plt.title("Actual vs Predicted Sales (Test Set)")
    plt.xlabel("Actual Sales")
    plt.ylabel("Predicted Sales")
    plt.tight_layout()
    # plt.savefig('actual_vs_predicted_test.png')

    # Plot error distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(test_results["error"], kde=True)
    plt.title("Error Distribution")
    plt.xlabel("Prediction Error")
    plt.tight_layout()
    # plt.savefig('error_distribution.png')

    return test_mae, test_rmse, test_wape, test_preds, y_test, test_results

In [20]:
# Prophet Model Results:
# MAE: 9.03 | RMSE: 11.86 | WAPE: 29.13%

In [21]:
# Evaluate the lightgbm model
test_mae, test_rmse, test_smape, test_preds, y_test_values, test_results = (
    evaluate_model(lightgbm_model, X_test, y_test, df_features)
)

NameError: name 'lightgbm_model' is not defined

## Save trained models

In [ ]:
def save_model(model, X_train, feature_names, output_dir="../models"):
    """
    Save the trained model and related artifacts for API use

    Args:
        model: Trained model (e.g., LightGBM model)
        feature_names: List of feature names
        output_dir: Directory to save model artifacts
    """

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the model
    model_path = os.path.join(output_dir, "sales_forecast_model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    print(f"Model saved to {model_path}")

    # Create and save feature statistics
    feature_stats = {
        "model_version": "1.0.0",
        "last_trained": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "required_columns": list(feature_names),
        "column_order": list(feature_names),
        "default_values": {},
        "temperature_bins": [-np.inf, 20, 25, 30, np.inf],
        "temperature_labels": ["Cold", "Cool", "Warm", "Hot"],
        "humidity_bins": [-np.inf, 60, 75, np.inf],
        "humidity_labels": ["Low", "Medium", "High"],
    }

    # Add default values for date features
    feature_stats["default_values"] = {
        "year": 2017,
        "month": 11,
        "day": 15,
        "day_of_week": 2,
        "is_weekend": 0,
        "quarter": 4,
        "is_holiday": 0,
    }

    # Save feature stats
    stats_path = os.path.join(output_dir, "feature_stats.json")
    with open(stats_path, "w") as f:
        json.dump(feature_stats, f, indent=4)
    print(f"Feature statistics saved to {stats_path}")

    print(f"All model artifacts saved successfully to {output_dir}/")

    return model_path, stats_path

In [ ]:
# Save model
save_model(
    model=optimized_model,
    X_train=X_train,
    feature_names=X_train.columns,
    output_dir='../models'
)

Model saved to ../models/sales_forecast_model.pkl
Feature statistics saved to ../models/feature_stats.json
All model artifacts saved successfully to ../models/


('../models/sales_forecast_model.pkl', '../models/feature_stats.json')